In [1]:
import spacy
from spacy import displacy
from tqdm import tqdm

nlp = spacy.load("en_core_web_sm")

C:\Users\Varun\ATAI\venv\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\Varun\ATAI\venv\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\Varun\ATAI\venv\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\Varun\ATAI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [3]:
doc = nlp("Telsa Inc is going to aquire Twitter for 45 billion dollars in 2023")

for ent in doc.ents:
    print(ent.text,"|",ent.label_)
    

Telsa Inc | ORG
Twitter | PRODUCT
45 billion dollars | MONEY
2023 | DATE


In [4]:
doc = nlp("Who is the director of Batman 1989")

for ent in doc.ents:
    print(ent.text,"|",ent.label_)
    

displacy.render(doc, style = "ent")

Batman 1989 | ORG


In [5]:
from rdflib.namespace import Namespace, RDF, RDFS, XSD
from rdflib.term import URIRef, Literal
import csv
import json
import networkx as nx
import pandas as pd
import rdflib
from collections import defaultdict, Counter
import locale

In [6]:
_ = locale.setlocale(locale.LC_ALL, '')
from _plotly_future_ import v4_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = 'jupyterlab+svg'

In [7]:
graph = rdflib.Graph()
graph.parse('./14_graph.nt', format='turtle')

<Graph identifier=Nd91a4e8e1f8f4239912677121340138b (<class 'rdflib.graph.Graph'>)>

In [8]:
# prefixes used in the graph
WD = Namespace('http://www.wikidata.org/entity/')
WDT = Namespace('http://www.wikidata.org/prop/direct/')
SCHEMA = Namespace('http://schema.org/')
DDIS = Namespace('http://ddis.ch/atai/')

In [9]:
entities = set(graph.subjects()) | {s for s in graph.objects() if isinstance(s,URIRef)}
predicates = set(graph.predicates())
literals = {s for s in graph.objects() if isinstance(s, Literal)}
with_type = set(graph.subjects(WDT['P31'], None))
with_super = set(graph.subjects(WDT['P279'], None))
types = set(graph.objects(None, WDT['P31']))
supers = set(graph.objects(None, WDT['P279']))
with_label = set(graph.subjects(RDFS.label, None))
n_ents = len(entities)
n_rels = len(predicates)
n_lits = len(literals)
t_tot = len(graph)
t_ent = len([1 for s,p,o in graph.triples((None, None, None)) if isinstance(o,URIRef)])
t_lit = t_tot - t_ent
n_notype = len(entities - with_type - with_super)
n_notype_flt = len(entities - with_type - with_super - types - supers)

In [10]:


count = 1000
for predicate in predicates:
    count-=1
    if(count==0):
        break
    print(str(predicate))
    


http://www.wikidata.org/prop/direct/P3092
http://www.wikidata.org/prop/direct/P2061
http://www.wikidata.org/prop/direct/P3301
http://www.wikidata.org/prop/direct/P5096
http://www.wikidata.org/prop/direct/P1001
http://www.wikidata.org/prop/direct/P6216
http://www.wikidata.org/prop/direct/P1433
http://www.wikidata.org/prop/direct/P166
http://www.wikidata.org/prop/direct/P5201
http://www.wikidata.org/prop/direct/P1412
http://www.wikidata.org/prop/direct/P407
http://www.wikidata.org/prop/direct/P3095
http://www.wikidata.org/prop/direct/P8411
http://www.wikidata.org/prop/direct/P8670
http://www.wikidata.org/prop/direct/P1037
http://www.wikidata.org/prop/direct/P131
http://www.wikidata.org/prop/direct/P2348
http://www.wikidata.org/prop/direct/P828
http://www.wikidata.org/prop/direct/P4437
http://www.wikidata.org/prop/direct/P2756
http://www.wikidata.org/prop/direct/P275
http://www.wikidata.org/prop/direct/P767
http://www.wikidata.org/prop/direct/P1455
http://www.wikidata.org/prop/direct/P384

In [11]:
genre_list = {str(s):"<"+str(uri)+">" for uri,s in graph.query('''
PREFIX ddis: <http://ddis.ch/atai/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX schema: <http://schema.org/>
SELECT ?genre ?lbl WHERE {
?movie wdt:P31 wd:Q11424 .
?movie wdt:P136 ?genre .
?genre rdfs:label ?lbl .
}
''')}

print(genre_list)


{'LGBT-related film': '<http://www.wikidata.org/entity/Q20442589>', 'romance film': '<http://www.wikidata.org/entity/Q1054574>', 'comedy film': '<http://www.wikidata.org/entity/Q157443>', 'Western film': '<http://www.wikidata.org/entity/Q172980>', 'parody film': '<http://www.wikidata.org/entity/Q622548>', 'drama': '<http://www.wikidata.org/entity/Q130232>', 'action film': '<http://www.wikidata.org/entity/Q188473>', 'thriller film': '<http://www.wikidata.org/entity/Q2484376>', 'kung fu film': '<http://www.wikidata.org/entity/Q3072042>', 'war film': '<http://www.wikidata.org/entity/Q369747>', 'documentary film': '<http://www.wikidata.org/entity/Q93204>', 'gangster film': '<http://www.wikidata.org/entity/Q7444356>', 'historical film': '<http://www.wikidata.org/entity/Q17013749>', 'biographical film': '<http://www.wikidata.org/entity/Q645928>', 'crime thriller': '<http://www.wikidata.org/entity/Q19367312>', 'film based on books': '<http://www.wikidata.org/entity/Q52207310>', 'spy film': '<

In [12]:
genre_df = pd.DataFrame(genre_list.items(),columns=["genre","uri"])
genre_df.to_pickle("./genres.pkl")

In [13]:
people_list = {str(s):"<"+str(uri)+">" for uri,s in graph.query('''
PREFIX ddis: <http://ddis.ch/atai/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX schema: <http://schema.org/>
SELECT ?person ?lbl WHERE {
?person wdt:P31 wd:Q5 .
?person rdfs:label ?lbl .
}
''')}

len(people_list)

98653

In [14]:
name_df = pd.DataFrame(people_list.items(),columns=["name","uri"])
name_df.to_pickle("./names.pkl")

In [15]:
movie_dict = {str(s):"<"+str(uri)+">" for uri,s in graph.query('''
PREFIX ddis: <http://ddis.ch/atai/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX schema: <http://schema.org/>
SELECT ?movie ?lbl WHERE {
?movie wdt:P31 wd:Q11424 .
?movie rdfs:label ?lbl .
}
''')}

len(movie_dict)

23065

In [16]:
movie_df = pd.DataFrame(movie_dict.items(),columns=["movie_name","uri"])
movie_df.to_pickle("./movies.pkl")

In [17]:
#predicate = 'http://www.wikidata.org/prop/direct/P366'
predicate_dict = {}
prefix_string = "PREFIX ddis: <http://ddis.ch/atai/> PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX schema: <http://schema.org/> "
for uri in predicates:
    try:
        p = uri.split("/")[-1]
        query_string = "SELECT ?lbl WHERE { wdt:PREDICATE rdfs:label ?lbl . }".replace("PREDICATE",p)
        res = graph.query(prefix_string + query_string)
        label, = res
        predicate_dict[str(label[0])] = "<"+str(uri)+">"
    except:
        pass


predicate_dict

{'film crew member': '<http://www.wikidata.org/prop/direct/P3092>',
 'aspect ratio': '<http://www.wikidata.org/prop/direct/P2061>',
 'broadcast by': '<http://www.wikidata.org/prop/direct/P3301>',
 'member of the crew of': '<http://www.wikidata.org/prop/direct/P5096>',
 'applies to jurisdiction': '<http://www.wikidata.org/prop/direct/P1001>',
 'copyright status': '<http://www.wikidata.org/prop/direct/P6216>',
 'published in': '<http://www.wikidata.org/prop/direct/P1433>',
 'award received': '<http://www.wikidata.org/prop/direct/P166>',
 'IMDA rating': '<http://www.wikidata.org/prop/direct/P5201>',
 'languages spoken, written or signed': '<http://www.wikidata.org/prop/direct/P1412>',
 'language of work or name': '<http://www.wikidata.org/prop/direct/P407>',
 'practiced by': '<http://www.wikidata.org/prop/direct/P3095>',
 'set in environment': '<http://www.wikidata.org/prop/direct/P8411>',
 'character designer': '<http://www.wikidata.org/prop/direct/P8670>',
 'director / manager': '<http:

In [18]:
result = graph.query('''
PREFIX ddis: <http://ddis.ch/atai/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX schema: <http://schema.org/>
SELECT ?movie ?lbl ?rating WHERE {
?movie wdt:P31 wd:Q11424 .
?movie ddis:rating ?rating .
?movie rdfs:label ?lbl .
}
''')

In [19]:
for s in result:
    print(str(s))

(rdflib.term.URIRef('http://www.wikidata.org/entity/Q1004657'), rdflib.term.Literal('Bully', lang='en'), rdflib.term.Literal('6.9', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q1018729'), rdflib.term.Literal('The Chinese Feast', lang='en'), rdflib.term.Literal('6.9', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q1022401'), rdflib.term.Literal('The Super', lang='en'), rdflib.term.Literal('5.7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q1024694'), rdflib.term.Literal('Nightbreed', lang='en'), rdflib.term.Literal('6.6', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q1025198'), rdflib.term.Literal('Cadillac Man', lang='en'), rdflib.term.Literal('5.7', datatype=rdflib.

(rdflib.term.URIRef('http://www.wikidata.org/entity/Q2363894'), rdflib.term.Literal('In the Soup', lang='en'), rdflib.term.Literal('7.1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q2363955'), rdflib.term.Literal('The Last American Virgin', lang='en'), rdflib.term.Literal('6.3', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q2364339'), rdflib.term.Literal('Power', lang='en'), rdflib.term.Literal('5.7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q2364759'), rdflib.term.Literal('Cobb', lang='en'), rdflib.term.Literal('6.4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q2365654'), rdflib.term.Literal('The Experts', lang='en'), rdflib.term.Literal('4.8', datatype=rdfli

(rdflib.term.URIRef('http://www.wikidata.org/entity/Q932717'), rdflib.term.Literal('Tashan', lang='en'), rdflib.term.Literal('3.9', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q934960'), rdflib.term.Literal('Still Crazy', lang='en'), rdflib.term.Literal('7.1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q93512'), rdflib.term.Literal('Water Lilies', lang='en'), rdflib.term.Literal('6.7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q938294'), rdflib.term.Literal('Buddy', lang='en'), rdflib.term.Literal('6.8', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q938790'), rdflib.term.Literal('My Giant', lang='en'), rdflib.term.Literal('5.0', datatype=rdflib.term.URIRef('http

(rdflib.term.URIRef('http://www.wikidata.org/entity/Q1238102'), rdflib.term.Literal('Dominion: Prequel to the Exorcist', lang='en'), rdflib.term.Literal('5.2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q1254081'), rdflib.term.Literal('Mondays in the Sun', lang='en'), rdflib.term.Literal('7.6', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q1254460'), rdflib.term.Literal('House of Dracula', lang='en'), rdflib.term.Literal('5.8', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q1334314'), rdflib.term.Literal('Elvira, Mistress of the Dark', lang='en'), rdflib.term.Literal('6.6', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://www.wikidata.org/entity/Q1346745'), rdflib.term.Literal('If....', lang='e

In [20]:
import editdistance

In [73]:



entity = "Cars"

question = f"who is the cast mmber of {entity}?"

#roles = ["actor","director","producer"]

In [74]:
import re

# a naive way for matching entities and relations

question_pattern = "who is the (.*) of ENTITY"

print("question pattern: {}\n".format(question_pattern))

question = re.sub(entity, "ENTITY", question.rstrip("?"))  # preprocess the question

relation = re.match(question_pattern, question).group(1)  # match the relation using a pattern

print("recognized relation: {}\n".format(relation))

question pattern: who is the (.*) of ENTITY

recognized relation: cast mmber



In [75]:
from rapidfuzz import process,fuzz

def match_entity(entity,entity_dict):
    
    result = None
    substr_result = None
    for key,val in entity_dict.items():
        if(entity.lower()==key.lower() and result==None):
            result=val
            print("exact match")
        elif(entity.lower() in key.lower() and substr_result==None):
            substr_result = val
            print("substring match")
    
    if(result!=None):
        return result
    if(substr_result!=None):
        return substr_result
    
    print("fuzzy match")
            
    
    fuzz_match = process.extract(entity, entity_dict.keys(), scorer=fuzz.WRatio, limit=3)
    print(fuzz_match)
    return entity_dict[fuzz_match[0][0]]

    """
    tmp = 9999
    match_node = ""
    #print("--- entity matching for \"{}\"\n".format(entity))
    for key, value in entity_dict.items():
        #print("edit distance between {} and {}: {}".format(value, entity, editdistance.eval(value, entity)))
        if editdistance.eval(key, entity) < tmp:
            tmp = editdistance.eval(key, entity)
            match_node = value
    
            
    return match_node"""

match_node = match_entity(entity,movie_dict)
#match_pred = match_entity(relation,predicate_list)

print("\n--- the matching node of \"{}\" is {}\n".format(entity, match_node))
#print("--- the matching predicates of \"{}\" is {}\n".format(relation, match_pred))

substring match

--- the matching node of "Cars" is <http://www.wikidata.org/entity/Q1313530>



In [76]:
query_string = "SELECT ?y WHERE { <MOVIE_NAME> <PREDICATE> ?x . ?x rdfs:label ?y}".replace("PREDICATE",match_pred).replace("MOVIE_NAME",match_node)
constructed_query = prefix_string + query_string
print(constructed_query)
res = graph.query(constructed_query)

for s in res:
    print(str(s[0]))

NameError: name 'match_pred' is not defined

In [77]:
# Defining some dictionaries for mapping wrods to predicates
roles = {
    "actor":"P161",
    "cast":"P161",
    "cast member":"P161",
    "director":"P57",
    "screenwriter":"P58",
    "producer":"P162",
}

actions = {
    "acted":"P161",
    "directed":"P57",
    "screenwrote":"P58",
    "wrote":"P58",
    "written":"P58",
    "produced":"P162",
    "featured":"P161",
    "featuring":"P161",    
}

roles_dict = {k:"<http://www.wikidata.org/prop/direct/"+v+">" for k,v in roles.items()}
actions_dict = {k:"<http://www.wikidata.org/prop/direct/"+v+">" for k,v in actions.items()}


genres = {v.replace(" film",""):k for k,v in genre_list.items()}

actions_dict

{'acted': '<http://www.wikidata.org/prop/direct/P161>',
 'directed': '<http://www.wikidata.org/prop/direct/P57>',
 'screenwrote': '<http://www.wikidata.org/prop/direct/P58>',
 'wrote': '<http://www.wikidata.org/prop/direct/P58>',
 'written': '<http://www.wikidata.org/prop/direct/P58>',
 'produced': '<http://www.wikidata.org/prop/direct/P162>',
 'featured': '<http://www.wikidata.org/prop/direct/P161>',
 'featuring': '<http://www.wikidata.org/prop/direct/P161>'}

In [78]:

sentence_list_1 = [
    "Who is the <role> of <movie>?",
    "Who are the <role>s of <movie>?",
    "Who <directed/wrote> <movie>? ",
    "Who acted in <movie>?",
    "Who was the <role> of <movie>?",
    "Give me the <role> of <movie>",
    "Tell me the <role> for <movie>",
    "Who <directed/wrote> the movie <movie>?",
    "Identify the <role> of <movie>",
    "I'd like to know the <role>'s name for <movie>",
    "Who was in charge of directing <movie>?",
    "Give me the name of the <role> of <movie>",
    "What is the <role>'s name for the movie <movie>?",
    "Tell me the <role> behind <movie>",
    "Who is the <role> of the film <movie>?",
    "I'm curious about the person who <directed/wrote> <movie>"

]

query_1 = "SELECT ?y WHERE { <movie> <action/role> ?x . ?x rdfs:label ?y}"


sentence_list_2 = [
    "List movies directed by <name>",
    "List movies written by <name>",
    "Give me a list of movies featuring <name>",
    "Give me a list of movies directed by <name>",
    "Give me a list of <genre> movies directed by <name>",
    "Give me a list of <genre> movies featuring <name>",
    "Give me a list of <genre> movies written by <name>",
    "Give me a list of movies written by <name>",
    "Which movies has <name> directed?",
    "Which movies has <name> <acted/wrote>"
]

query_2 = "SELECT ?y WHERE { ?x <action/role> <name> . ?x rdfs:label ?y . ?x wdt:P136 <genre> . }"


sentence_list_3 = [
    "When was <movie> released?",
    "When did <movie> come out?",
    "Tell me the release date of <movie>",
    "What is the release year of <movie>?",
    "When did <movie> come out?",
    "Give me the release date for <movie>",
    "What's the release year of the movie <movie>?",
    "I'd like to know the release date of the movie <movie>.",
    "When was the movie <movie> released?",
    "Please provide the release year for <movie>",
    "What's the date of release for <movie>?",
    "When did <movie> hit the theaters?"
]

query_3 = "SELECT ?x WHERE { <movie> wdt:P577 ?x}"


sentence_list_4 = [
    "Which genre does the movie <movie> belong to?",
    "Which genre does <movie> belong to?",
    "Tell me the genre of <movie>",
    "What genre is associated with <movie>?",
    "Identify the movie genre for <movie>",
    "I'd like to know the genre of <movie>",
    "What category does <movie> fall into?",
    "Give me the genre for the movie <movie>",
    "What is the movie genre for <movie>?",
    "Tell me the type of film that is <movie>",
    "Which category does <movie> fit into?",
    "I'm curious about the genre of <movie>"
]

query_4 = "SELECT ?y WHERE { <movie> wdt:P136 ?x . ?x rdfs:label ?y}"

sentence_list_5 = [
    "Tell me the movie with the <highest/lowest> user rating",
    "What movie has the <highest/lowest> user rating?",
    "Identify the film with the <highest/lowest> user rating",
    "I'd like to know the movie with the <highest/lowest> user rating",
    "Which film received the <highest/lowest> user rating?",
    "Give me the movie that has the <highest/lowest> user rating",
    "What is the movie with the <highest/lowest> user rating?",
    "Tell me the film that received the <highest/lowest> user rating",
    "Which movie is rated the <highest/lowest> by users?",
    "I'm curious about the movie with the <highest/lowest> user rating",

    "Show me the <highest/lowest>-rated films.",
    "List the movies with the <best/worst> ratings.",
    "I want to see the <top/bottom>-rated movies.",
    "Display the movies with the <highest/lowest> ratings.",
    "What are the most highly-rated films?",
    "Provide a list of the <top/bottom>-rated movies.",
    "Give me the <highest/lowest>-rated films.",
    "Show the movies with the <best/worst> ratings",
    "Retrieve the movies with the <highest/lowest> ratings",]

query_5 = "SELECT ?lbl WHERE { SELECT ?movie ?lbl ?rating WHERE { ?movie wdt:P31 wd:Q11424 . ?movie ddis:rating ?rating . ?movie rdfs:label ?lbl . } ORDER BY <order>(?rating) LIMIT <number>}"

sentence_list_6 = [
    "Show me the <highest/lowest>-rated <genre> films",
    "List the <genre> movies with the <best/worst> ratings",
    "I want to see the <top/bottom>-rated <genre> movies",
    "Display the <genre> movies with the <highest/lowest> ratings",
    "What are the most highly-rated <genre> films?",
    "Provide a list of the <top/bottom>-rated <genre> movies",
    "Give me the <highest/lowest>-rated <genre> films",
    "Show the movies with the <best/worst> <genre> ratings",
    "Retrieve the movies with the <highest/lowest> <genre> ratings",
]

query_6 = "SELECT ?lbl WHERE { SELECT ?movie ?lbl ?rating WHERE { ?movie wdt:P31 wd:Q11424 . ?movie ddis:rating ?rating . ?movie wdt:P136 <genre> . ?movie rdfs:label ?lbl . } ORDER BY <order>(?rating) LIMIT <number>}"

sentence_list_7 = [
    "Show me the <highest/lowest>-rated <genre> films <action> <name>",
    "List the <genre> movies with the <best/worst> ratings <action> <name>",
    "I want to see the <top/bottom>-rated <genre> movies <action> <name>",
    "Display the <genre> movies with the <highest/lowest> ratings <action> <name>",
    "What are the most highly-rated <genre> films <action> <name>?",
    "Provide a list of the <top/bottom>-rated <genre> movies <action> <name>",
    "Give me the <highest/lowest>-rated <genre> films <action> <name>",
    "Show the movies with the <best/worst> <genre> ratings <action> <name>",
    "Retrieve the movies with the <highest/lowest> <genre> ratings <action> <name>",

    "List the <genre> movies <action> <name> with the <best/worst> ratings",
    "Display the <genre> movies <action> <name> with the <highest/lowest> ratings",
    "Provide a list of the <top/bottom>-rated <genre> movies",
    "Show the movies <action> <name> with the <best/worst> <genre> ratings",
    "Retrieve the movies <action> <name> with the <highest/lowest> <genre> ratings",
]

query_7 = "SELECT ?lbl WHERE { SELECT ?movie ?lbl ?rating WHERE { ?movie wdt:P31 wd:Q11424 . ?movie ddis:rating ?rating . ?movie <action> <name> . ?movie wdt:P136 <genre> . ?movie rdfs:label ?lbl . } ORDER BY <order>(?rating) LIMIT <number>}"

sentence_list_8 = [ # year-related
    "List the movies released in <year>",
    "Which movies were directed by <name> in <year>?",
    "What movies were released in <year>?",
    "What movies did <name> appear in in <year>?",
    "List the films featuring <name> in <year>",
    "Provide the filmography of <name> in <year>",
    "Which movies starred <name> in <year>?",
    "Which roles were played by <name> in <year>?"
]

query_8 = 'SELECT ?lbl  WHERE { <movie> wdt:P31 wd:Q11424 . ?movie wdt:P577 ?release_date ?movie rdfs:label ?lbl .FILTER regex(str(?release_date), "<year>") .}'


sentence_list_9 = [ #Top/lowest N (based on release date)
    "What are the <best/worst> <number> movies <name> directed?",
    "List the <best/worst> <number> movies directed by <name>",
    "What are the <best/worst> <number> movies <name> directed?",
    "What are the <best/worst> <number> <genre> movies <name> directed?",
    "Which are the <best/worst> movies where <name> appeared?",
    "Can you name the <best/worst> <number> <genre> movies directed by <name>?",
    "Can you list the <best/worst> <number> films with <name> in the cast?",
    "What are the <best/worst> <number> movies starring <name>"
]

query_9 = "SELECT ?lbl WHERE { SELECT ?movie ?lbl ?rating WHERE { ?movie wdt:P31 wd:Q11424 . ?movie ddis:rating ?rating . ?movie <action> <name> . ?movie wdt:P136 <genre> . ?movie rdfs:label ?lbl . } ORDER BY <order>(?rating) LIMIT <number>}"

sentence_list_10 = [ #Yes/No
    "Is <name> in the cast of <movie>?",
    "Did <name> direct <movie>?",
    "Was the <movie> released in <year>?",
    "Is <name> part of the cast of <movie>?",
    "Was <movie> released before <year>?",
    "Is <movie> classified as <genre>?",
    "Did <name> appear in <movie>?",
]

query_10 = 'ASK {<name> <action> <movie> . <movie> wdt:P577 ?release_date .<movie> wdt:P136 <genre> FILTER regex(str(?release_date), "<year>") . }'

sentence_list_11 = [
        
    # genre
    "Recommend me movies with the genres <genre>, <genre>, and <genre>",
    "Are there any <genre> and <genre> movies?",
    "I am looking for <genre> and <genre> movies, anything you would recommend?",
    "Are there any movies that blend the genres of <genre> and <genre>?",
    "I am looking for movies that fall under both <genre> and <genre>. Anything you would recommend?",
    "Can you suggest films that seamlessly combine elements of <genre> and <genre>?",
    "Which movies successfully incorporate both <genre> and <genre> in their storytelling?",
    "Do you have recommendations for movies that showcase a mix of <genre> and <genre>?",
    "I'm interested in exploring the intersection of <genre> and <genre>. Any standout films in that category?",
    "Can you provide a list of movies that are known for their unique blend of <genre> and <genre>?",
    "Are there any critically acclaimed movies that successfully navigate the genres of both <genre> and <genre>?",
    "I enjoy movies that combine the elements of <genre> and <genre>. What are your top picks in this category?",
    "Can you recommend movies that span a wide range of genres, including <genre>, <genre>, and <genre>?",
    "I'm in the mood for a movie night with diverse genres. Any suggestions for films that include <genre> and <genre>?",
    "Do you have recommendations for movies that masterfully balance the genres of both <genre> and <genre>?"
    
]

sentence_list_12 = [
    # Movie recommendation
    # Three movies
    "Given that I like <movie>, <movie>, and <movie>, can you recommend some movies?",
    "Recommend movies like <movie>, <movie>, and <movie>",
    "Since I enjoy <movie>, <movie>, and <movie>, do you have any movie recommendations for me?",
    "I'm a fan of <movie>, <movie>, and <movie>. Any suggestions for similar movies?",
    "Considering my liking for <movie>, <movie>, and <movie>, could you suggest some other films I might enjoy?",
    "Given my interest in <movie>, <movie>, and <movie>, can you recommend additional movies for me to watch?",
    "If I enjoy <movie>, <movie>, and <movie>, what other movies would you suggest?",
    "Being a fan of <movie>, <movie>, and <movie>, are there similar movies you'd recommend?",
    "Since I like <movie>, <movie>, and <movie>, what other movies do you think I might like?",
    "I've enjoyed <movie>, <movie>, and <movie>. Any recommendations for movies in a similar vein?",
    "Given my preference for <movie>, <movie>, and <movie>, what other films should I check out?",
    "If <movie>, <movie>, and <movie> are my preferences, do you have any movie recommendations for me?",
    "Suggest films similar to <movie>, <movie>, and <movie>",
    "Can you recommend movies akin to <movie>, <movie>, and <movie>?",
    "Looking for movies in the same vein as <movie>, <movie>, and <movie>. Any recommendations?",
    "Seeking recommendations for movies like <movie>, <movie>, and <movie>",
    "Do you know any movie similar to <movie>, <movie>, and <movie>",
    "I'm on the lookout for captivating films that share a vibe with <movie>, <movie>, and <movie>. Any suggestions that align with these favorites of mine?",
    "In the mood for something reminiscent of <movie>, <movie>, and <movie>. Any movie recommendations that capture a similar essence?",
    "If my taste for movies like <movie>, <movie>, and <movie> is any indication, what other films should I consider watching?",
    "Looking to expand my movie playlist based on the likes of <movie>, <movie>, and <movie>. Any hidden gems you'd recommend?",
    "If the cinematic style of <movie>, <movie>, and <movie> resonates with you, could you suggest other films that share a similar appeal?",
    "Enjoyed the storytelling in <movie>, <movie>, and <movie>. Any recommendations for movies that weave narratives in a comparable fashion?",
    "Based on the cinematic experience of <movie>, <movie>, and <movie>, are there any other films you'd suggest to keep the excitement alive?",
    "If the movie magic of <movie>, <movie>, and <movie> has left a lasting impression, what other films do you think would leave me equally enchanted?",
    "Immersed in the world of <movie>, <movie>, and <movie>, I'm curious about your recommendations for movies that provide a similar level of immersion.",
    "Exploring films akin to <movie>, <movie>, and <movie> — any suggestions to keep the cinematic journey going?",
    "Captivated by the storytelling in <movie>, <movie>, and <movie>. What other films do you think would tell a tale in a way that resonates with me?",
    
    
    
    #Two movies
    "Being a fan of <movie> and <movie>, are there similar movies you'd recommend?",
    "Based on the cinematic experience of <movie> and <movie>, are there any other films you'd suggest to keep the excitement alive?",
    "Can you recommend movies akin to <movie> and <movie>?",
    "Captivated by the storytelling in <movie> and <movie>. What other films do you think would tell a tale in a way that resonates with me?",
    "Considering my liking for <movie> and <movie>, could you suggest some other films I might enjoy?",
    "Do you know any movie similar to <movie> and <movie>?",
    "Enjoyed the storytelling in <movie> and <movie>. Any recommendations for movies that weave narratives in a comparable fashion?",
    "Given my interest in <movie> and <movie>, can you recommend additional movies for me to watch?",
    "Given my preference for <movie> and <movie>, what other films should I check out?",
    "I'm a fan of <movie> and <movie>. Any suggestions for similar movies?",
    "If I enjoy <movie> and <movie>, what other movies would you suggest?",
    "I've enjoyed <movie> and <movie>. Any recommendations for movies in a similar vein?",
    "Looking for movies in the same vein as <movie> and <movie>. Any recommendations?",
    "Recommend movies like <movie> and <movie>",
    "Seeking recommendations for movies like <movie> and <movie>",
    "Since I enjoy <movie> and <movie>, do you have any movie recommendations for me?",
    "Since I like <movie> and <movie>, what other movies do you think I might like?",
    "Suggest films similar to <movie> and <movie>",
    "If <movie> and <movie> are my preferences, do you have any movie recommendations for me?",
    "Can you recommend movies that share a similar theme or storyline with both <movie> and <movie>?",
    
    #One movie
    "Since I enjoy <movie>, do you have any movie recommendations for me?",
    "Recommend movies like <movie>",
    "I'm a fan of <movie>. Any suggestions for similar movies?",
    "Considering my liking for <movie>, could you suggest some other films I might enjoy?",
    "Given my interest in <movie>, can you recommend additional movies for me to watch?",
    "If I enjoy <movie>, what other movies would you suggest?",
    "Being a fan of <movie>, are there similar movies you'd recommend?",
    "Since I like <movie>, what other movies do you think I might like?",
    "I've enjoyed <movie>. Any recommendations for movies in a similar vein?",
    "Given my preference for <movie>, what other films should I check out?",
    "If <movie> is my preference, do you have any movie recommendations for me?",
    "Suggest films similar to <movie>",
    "Can you recommend movies akin to <movie>?",
    "Looking for movies in the same vein as <movie>. Any recommendations?",
    "Seeking recommendations for movies like <movie>",

]

# Repeated instances
sentence_list_13 = [
    # Names
    "Do <name> and <name> appear in <movie>",
    "Do <name> and <name> appear in the same movie",
    "Have <name> and <name> ever worked together?",
    "Can you list all the movies starring <name>, <name>, and <name>?",
    "Can you recommend me a movie with a similar style than the movies from <name> and <name>", 
    "Can you list all the movies starring both <name> and <name>?",
    "Are there any films where <name>, <name> and <name> collaborated as <role>",
    "Is there a movie that features both <name> and <name> in prominent roles?",
    "Have <name> and <name> ever co-directed movies?",
    "Do <name> and <name> share the screen in any movies?",
    "Are there any notable collaborations between <name>, <name>, and <name> in the film industry?",
    "Do <name> and <name> have any projects together?",
    "Can you recommend a movie that showcases the directorial styles of both <name> and <name>?",
    "Can you provide a list of films directed by <name> and starring <name>?",
    "Have <name> and <name> ever been involved in a movie project that received significant critical acclaim?",
    
    # Movies
    "Are there any staff members in common between <movie> and <movie>?",
    "Were <movie> and <movie> released the same year?",
    "Can you suggest films where the cast includes both <name> and <name>?",
    "Do <movie> and <movie> share any crew members or production staff?",
    "Which actors appeared in both <movie> and <movie>?",
    "Are there any common themes or genres between <movie> and <movie>?",
    "Do <movie> and <movie> belong to the same film franchise?",
    "Are there any notable connections between <movie> and <movie> in terms of directors or producers?",
    "Which actors from <movie> and <movie> have collaborated in other films?",
    "Are there any common filming locations between <movie> and <movie>?",
    "Which <role> or <role> contributed to the soundtracks of both <movie> and <movie>?",
]



query_list = [query_1,query_2,query_3,query_4,query_5,query_6,query_7,query_8,query_9,query_10]

In [79]:
# (text,{"entities":[(start,end,entity)]})
i = 0
peoples = list(people_list.values())
genres = list(genre_list.values())
res_s2 = []
for name in peoples:
    i+=1
    if(i==10000):
        break
        
    genre = genres[i%len(genres)]
        
    sentence = sentence_list_2[i%len(sent
                                     ence_list_2)]
    
    if(i%2==0):
        AoW = "acted" 
    else: 
        AoW = "wrote"
    sentence2 = sentence.replace("<name>",name).replace("<genre>",genre)
    sentence3 = sentence2.replace("<acted/wrote>",AoW)

    ents = [(sentence3.find(name),sentence3.find(name) + len(name), "name"),
            (sentence3.find(genre),sentence3.find(genre)+len(genre), "genre"),
            (sentence3.find(AoW),sentence3.find(AoW) + len(AoW), "action"),
            (sentence3.find("directed"),sentence3.find("directed") + len("directed"), "action"),
            (sentence3.find("written"),sentence3.find("written") + len("written"), "action"),
            (sentence3.find("featuring"),sentence3.find("featuring") + len("featuring"), "action")]

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)


    res_s2.append((sentence3,{"entities":entities}))
    



len(res_s2)
        
        
        

            

SyntaxError: invalid syntax. Perhaps you forgot a comma? (562578268.py, line 13)

In [80]:
res_s5 = []
i = 0
action_list = ["directed by","written by","featuring","produced by"]

sentence_list_567 = sentence_list_5 + sentence_list_6 + sentence_list_7
for name in peoples:
    i+=1
    if(i%2==0):
        HoL = "highest"
        ToB = "top"
        BoW = "best"
    else:
        HoL = "lowest"
        ToB = "bottom"
        BoW = "worst"
    
    action = action_list[i%len(action_list)]
    sentence = sentence_list_567[i%len(sentence_list_567)]
    genre = genres[i%len(genre_list)]

    sentence2 = sentence.replace("<highest/lowest>",HoL).replace("<top/bottom>",ToB).replace("<best/worst>",BoW)
    sentence3 = sentence2.replace("<name>",name).replace("<action>",action).replace("<genre>",genre)
    ents = [(sentence3.find(HoL),sentence3.find(HoL) + len(HoL), "order"),
            (sentence3.find("highly"),sentence3.find("highly") + len("highly"), "order"),
            (sentence3.find(ToB),sentence3.find(ToB)+len(ToB), "order"),
            (sentence3.find(genre),sentence3.find(genre) + len(genre), "genre"),
            (sentence3.find("directed"),sentence3.find("directed") + len("directed"), "action"),
            (sentence3.find("written"),sentence3.find("written") + len("written"), "action"),
            (sentence3.find("featuring"),sentence3.find("featuring") + len("featuring"), "action"),
            (sentence3.find("produced"),sentence3.find("produced") + len("produced"), "action"),
           ]


    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)

    res_s5.append((sentence3,{"entities":entities}))
    
    if(i==10000):
        break

(res_s5)


NameError: name 'peoples' is not defined

In [81]:
TRAIN_DATA = res_s1 + res_s2 + res_s5

len(TRAIN_DATA)

NameError: name 'res_s1' is not defined

In [82]:
import pandas as pd
import os
from tqdm import tqdm
from spacy.tokens import DocBin


In [83]:

nlp = spacy.load("en_core_web_sm")

db = DocBin() # create a DocBin object

count = 0
for text, annot in tqdm(TRAIN_DATA): # data in previous format
    try:
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                #print("Skipping entity")
                count+=1
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    except Exception as e:
        count+=1
print(count)
#os.chdir(r'XXXX\XXXXX')
db.to_disk("./train.spacy") # save the docbin object

NameError: name 'TRAIN_DATA' is not defined

In [ ]:
nlp1 = spacy.load(r"./models/NER/v3_165") #load the best model
nlp1.add_pipe("merge_entities")

In [ ]:
text = "Who directed the movie Batman"
def ner_inference(text,display=False):

    doc = nlp1(text) # input sample text
    
    if(display):
        spacy.displacy.render(doc, style="ent") # display in Jupyter
    preprocessed = " ".join([t.text if not t.ent_type_ else f"<{t.ent_type_.lower()}>" for t in doc])
    ent_dict = {t.text:f"<{t.ent_type_}>" for t in doc if t.ent_type}
    #print(ent_dict)
    
    return {"text":preprocessed,"entities":ent_dict}

In [69]:
res = ner_inference(text)
res["text"],res["entities"]

('Who <action> the movie <movie>',
 {'directed': '<action>', 'Batman': '<movie>'})

In [84]:
def replace_entity(sentence):
    ent_dict = {
        "<highest/lowest>":"<order>",
        "<top/bottom>":"<order>",
        "<best/worst>":"<order>",
        "<directed/wrote>":"<action>",
        "acted":"<action>",
        "highly":"<order>",
        "directed":"<action>",
        "produced":"<action>",
        "featuring":"<action>",
        "written":"<action>",
               }
    for key,value in ent_dict.items():
        sentence = sentence.replace(key,value)
    return sentence

sentence_lists = [sentence_list_1,sentence_list_2,sentence_list_3,sentence_list_4,sentence_list_5,sentence_list_6,sentence_list_7, sentence_list_8, sentence_list_9, sentence_list_12]

NUM_CLASSES = len(sentence_lists)
val_sentences = []
train_sentences = []
for i,sentence_list in enumerate(sentence_lists):
    sentence_list = [(replace_entity(s),i) for s in sentence_list]
    train_sentences+=sentence_list
    val_sentences+=sentence_list[:10]

train_df = pd.DataFrame(train_sentences,columns=["sentences","label"])
train_df

,sentences,label
0,Who is the <role> of <movie>?,0
1,Who are the <role>s of <movie>?,0
2,Who <action> <movie>?,0
3,Who <action> in <movie>?,0
4,Who was the <role> of <movie>?,0
...,...,...
166,"If <movie> is my preference, do you have any m...",9
167,Suggest films similar to <movie>,9
168,Can you recommend movies akin to <movie>?,9
169,Looking for movies in the same vein as <movie>...,9


In [85]:
max_len = 0

sentences = train_df["sentences"].to_list()
labels = train_df["label"].to_list()



# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

NameError: name 'tokenizer' is not defined

In [86]:
df = train_df

label_dict = {k:k for k in range(NUM_CLASSES)}

In [87]:

import string
import sys
import spacy

train_data = df.copy()

#train_data = train_data.drop("data_type",axis=1)

train_data = train_data.rename(columns={"sentences":"text"})



In [88]:
def make_docs(data_frame):
    """
    this will take a list of texts and labels 
    and transform them in spacy documents
    
    data: list(tuple(text, label))
    
    returns: List(spacy.Doc.doc)
    """
    train_data = data_frame
    train_data.dropna(axis = 0, how ='any',inplace=True) 
    train_data['Num_words_text'] = train_data['text'].apply(lambda x:len(str(x).split())) 
    mask = train_data['Num_words_text'] >2
    train_data = train_data[mask]
    print(train_data['label'].value_counts())
    
    #train_data['text'] = train_data['text'].apply(remove_emoji)
    #train_data['text'] = train_data['text'].apply(remove_url)
    #train_data['text'] = train_data['text'].apply(clean_text)
   
    data = tuple(zip(train_data['text'].tolist(), train_data['label'].tolist())) 
    print(data[1])
    docs = []
    # nlp.pipe([texts]) is way faster than running 
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple, 
    # the first one is treated as text
    # the second one will get returned as it is.
    nlp = spacy.load("en_core_web_trf")
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        
        # we need to set the (text)cat(egory) for each document
        #print(label)
            
        for i in range(NUM_CLASSES):
            if(label==i):
                doc.cats[str(i)]=1
            else:
                doc.cats[str(i)]=0
        #print(doc.cats)
        
        # put them into a nice list
        docs.append(doc)
    
    return docs,train_data

In [89]:
train_docs,train_data  = make_docs(train_data)
# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("../textcat/textcat_train.spacy")

label
9    63
4    19
0    16
6    14
2    12
3    12
1    10
5     9
7     8
8     8
Name: count, dtype: int64
('Who are the <role>s of <movie>?', 0)


100%|████████████████████████████████████████████████████████████████████████████████| 171/171 [00:13<00:00, 12.43it/s]


In [96]:
nlp_textcat = spacy.load("../textcat/textcat_output/model-best")




In [97]:
def textcat_inference(text):

    doc2 = nlp_textcat(text)

    max_val = -1000
    max_cat = -1

    #print(type(doc2.cats))

    for k,v in doc2.cats.items():
        if(v>max_val):
            max_val = v
            max_cat = k
            #print(k,v)

    return(max_cat)

In [98]:
for text,label in zip(train_data["text"],train_data["label"]):
    print(text,label,"->",textcat_inference(text))

Who is the <role> of <movie>? 0 -> 0
Who are the <role>s of <movie>? 0 -> 0
Who <action> <movie>?  0 -> 0
Who <action> in <movie>? 0 -> 0
Who was the <role> of <movie>? 0 -> 0
Give me the <role> of <movie> 0 -> 0
Tell me the <role> for <movie> 0 -> 0
Who <action> the movie <movie>? 0 -> 0
Identify the <role> of <movie> 0 -> 0
I'd like to know the <role>'s name for <movie> 0 -> 0
Who was in charge of directing <movie>? 0 -> 0
Give me the name of the <role> of <movie> 0 -> 0
What is the <role>'s name for the movie <movie>? 0 -> 0
Tell me the <role> behind <movie> 0 -> 0
Who is the <role> of the film <movie>? 0 -> 0
I'm curious about the person who <action> <movie> 0 -> 0
List movies <action> by <name> 1 -> 1
List movies <action> by <name> 1 -> 1
Give me a list of movies <action> <name> 1 -> 6
Give me a list of movies <action> by <name> 1 -> 1
Give me a list of <genre> movies <action> by <name> 1 -> 1
Give me a list of <genre> movies <action> <name> 1 -> 1
Give me a list of <genre> movies

Being a fan of <movie> and <movie>, are there similar movies you'd recommend? 9 -> 9
Based on the cinematic experience of <movie> and <movie>, are there any other films you'd suggest to keep the excitement alive? 9 -> 9
Can you recommend movies akin to <movie> and <movie>? 9 -> 9
Captivated by the storytelling in <movie> and <movie>. What other films do you think would tell a tale in a way that resonates with me? 9 -> 9
Considering my liking for <movie> and <movie>, could you suggest some other films I might enjoy? 9 -> 9
Do you know any movie similar to <movie> and <movie>? 9 -> 9
Enjoyed the storytelling in <movie> and <movie>. Any recommendations for movies that weave narratives in a comparable fashion? 9 -> 9
Given my interest in <movie> and <movie>, can you recommend additional movies for me to watch? 9 -> 9
Given my preference for <movie> and <movie>, what other films should I check out? 9 -> 9
I'm a fan of <movie> and <movie>. Any suggestions for similar movies? 9 -> 9
If I enjo

In [101]:
order_dict = {
    "highest":"DESC",
    "highly":"DESC",
    "best":"DESC",
    "top":"DESC",
    "lowest":"ASC",
    "worst":"ASC",
    "bottom":"ASC",
}

node_dict_map = {
    "<movie_name>":movie_dict,
    "<role>":roles_dict,
    "<action>":actions_dict,
    "<genre>":genre_list,
    "<name>":people_list,
    "<order>":order_dict,
                }

def inference(input_chat_text):
    ner_res = ner_inference(input_chat_text)
    print(ner_res)
    label = textcat_inference(ner_res["text"])

    print("LABEL: ",label)

    # TODO: Naive way to check if recommendation, need to make it as last label
    if(label=="9"):
        print("Recommendation")
        query_type = "REC"
        ent_list = []
        for entity,ent_type in ner_res["entities"].items():
            if(ent_type == "<name>" or ent_type == "<movie>"):
                ent_list.append(entity)

        return {"query_type": query_type,"entity_list":ent_list}

    else:
        query_type = "QUE"


        query = query_list[int(label)]
        s,p,o = query_spo[int(label)]

        print(ner_res["text"],label,ner_res["entities"])    
        
        #Replace entities in the sparQL query with the detected entities

        for entity,ent_type in ner_res["entities"].items():

            node_dict = node_dict_map[ent_type]
            print(ent_type,entity)
            matched_node = match_entity(entity,node_dict)

            if(ent_type == "<action>" or ent_type == "<role>"):
                ent_type = "<action/role>"
            s = s.replace(ent_type,matched_node)
            p = p.replace(ent_type,matched_node)
            o = o.replace(ent_type,matched_node)
            query = query.replace(ent_type,matched_node)

        #Default values if user hasn't provided values 
        query = query.replace("<number>","10")
        query = query.replace("<genre>","?genre")

        constructed_query = prefix_string + query

        return {"query_type":query_type,"query":constructed_query,"spo":(s,p,o)}


    #FIND EACH ENTITY IN PREDICATES AND USE EDIT DISTANCE TO FIND SUITABLE ONES

In [102]:
input_text = "Given that I like The Lion King, Pocahontas, and The Beauty and the Beast, can you recommend some movies?"
inference_res = inference(input_text)
inference_res

{'text': 'Given that I like <movie> , <movie> , and <movie> , can you <predicate> some movies ?', 'entities': {'The Lion King': '<movie>', 'Pocahontas': '<movie>', 'The Beauty and the Beast': '<movie>', 'recommend': '<predicate>'}}
LABEL:  9
Recommendation


{'query_type': 'REC',
 'entity_list': ['The Lion King', 'Pocahontas', 'The Beauty and the Beast']}

In [ ]:
inference_res["entity_list"]

In [ ]:
query_list